In [1]:
#!pip install dash
#!pip install xgboost

In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import numpy as np
import joblib  # cargar el modelo entrenado
import sklearn

In [3]:
print(sklearn.__version__)

1.5.2


In [5]:
app = dash.Dash()

In [7]:
# Cargar el modelo
modelo_diabetes = joblib.load('diabetes_NeuralNet.pkl')  

In [9]:
app.layout = html.Div([
    html.H1('Detección Riesgo de Diabetes', style={'textAlign': 'center', 'color': 'blue', 'fontSize': '36px',  'fontWeight': 'bold'}),
    html.H2('''Esta herramienta es una guía para indicar si el usuario está en riesgo de padecer o no diabetes 
    y en ningún caso reemplaza el diagnóstico médico.'''),
    
    html.Div([
        
        # Dropdown Estado General de Salud
        html.Label('En general, su estado de salud es:', style={'fontSize': 18, 'marginBottom': '20px', 'marginTop': '20px'}),  # Título
        dcc.Dropdown(
            id='GenHlth',
            options = [
                {'label':'Excelente', 'value':1}, {'label':'Muy bueno', 'value':2}, {'label':'Bueno', 'value':3},
                {'label':'Regular', 'value':4}, {'label':'Mala', 'value':5}
            ],
        style={'width': '300px', 'marginBottom': '20px', 'flex': 1} ), # Ancho del Dropdown 

        # Edad
        html.Label('Edad:', style={'fontSize': 18, 'marginBottom': '20px'}),
        dcc.Input(id='edad', type='number', step='any', placeholder='Ej: 37', min=18, 
                  style={'width': '300px', 'marginBottom': '20px', 'marginRight': '20px', 'flex': 1}),

        ]),
    html.Div([
        # Estatura
        html.Label('Estatura (en metros):', style={'fontSize': 18, 'marginBottom': '20px'}),
        dcc.Input(id='height', type='number', step='any', placeholder='Ej: 1.75', min=1, max=2.5, 
                  style={'width': '300px', 'marginBottom': '20px', 'marginRight': '20px'}),

        # Peso
        html.Label('Peso (en kg):', style={'fontSize': 18, 'marginBottom': '10px'}),
        dcc.Input(id='weight', type='number', step='any', placeholder='Ej: 70', min=10, max=200,   
                  style={'width': '300px', 'marginBottom': '20px'}),
    
        # Mostrar el resultado del IMC
        html.Div(id='BMI', style={'fontSize': 25, 'marginBottom': '20px'}),
        ]),
    html.Div([
        # Presión Arterial Alta
        html.Label('¿Tiene presión arterial alta?:', style={'fontSize': 18, 'marginBottom': '10px'}),
        dcc.RadioItems(
        id='HighBP',  
        options=[
            {'label': 'Sí', 'value': 1},  
            {'label': 'No', 'value': 0}  
        ],
        labelStyle={'display': 'block', 'marginBottom': '10px'} ),

        # Colesterol Alto
        html.Label('¿Tiene colesterol alto?:', style={'fontSize': 18, 'marginBottom': '10px'}),
        dcc.RadioItems(
        id='HighChol',  
        options=[
            {'label': 'Sí', 'value': 1},  
            {'label': 'No', 'value': 0}  
        ],
        labelStyle={'display': 'block', 'marginBottom': '10px'} ),

        # Ataque Cardíaco
        html.Label('¿Alguna vez ha tenido un ataque cardiaco?:', style={'fontSize': 18, 'marginBottom': '10px'}),
        dcc.RadioItems(
        id='HeartDiseaseorAttack',  
        options=[
            {'label': 'Sí', 'value': 1},  
            {'label': 'No', 'value': 0}  
        ],
        labelStyle={'display': 'block', 'marginBottom': '10px'} ),

         ]),

         # Contenedor para la predicción, centrado al final
        html.Div(id='prediccion_diabetes', style={'fontSize': 35, 'marginTop': '40px', 'textAlign': 'center', 'fontWeight': 'bold'})
        

        
])

    # rango de edad
def rango_edad(edad):
    if 18 <= edad <= 24:
        Age = 1 
    elif edad <= 29:
        Age = 2 
    elif edad <= 34:
        Age = 3  
    elif edad <= 39:
        Age = 4  
    elif edad <= 44:
        Age = 5   
    elif edad <= 49:
        Age = 6  
    elif edad <= 54:
        Age = 7  
    elif edad <= 59:
        Age = 8  
    elif edad <= 64:
        Age = 9  
    elif edad <= 69:
        Age = 10  
    elif edad <= 74:
        Age = 11  
    elif edad <= 79:
        Age = 12  
    else:
        Age = 13 
    return Age
        
# Callback para calcular el IMC/BMI
@app.callback(
    Output('BMI', 'children'),
    Output('prediccion_diabetes', 'children'),
    Input('height', 'value'),
    Input('weight', 'value'),
    Input('HighBP', 'value'),
    Input('HighChol', 'value'),
    Input('HeartDiseaseorAttack', 'value'),
    Input('GenHlth', 'value'),
    Input('edad', 'value')
)
def calcular_imc_and_predict(height, weight, HighBP, HighChol, HeartDiseaseorAttack, GenHlth, edad):
    if height and weight:
        try:
            # Calcular el IMC
            BMI = float(weight) / (float(height) ** 2)
            
            if BMI < 18.5:
                bmi_message = f"Su IMC es {BMI:.0f}: Bajo peso"
                color = 'green'
            elif BMI <= 24.9:
                bmi_message = f"Su IMC es {BMI:.0f}: Peso normal"
                color = '#FFCC00'
            elif BMI <= 29.9:
                bmi_message = f"Su IMC es {BMI:.0f}: Sobrepeso"
                color = 'orange'
            elif BMI <= 34.9:
                bmi_message = f"Su IMC es {BMI:.0f}: Obesidad"
                color = 'red'
            else:
                bmi_message = f"Su IMC es {BMI:.0f}: Obesidad mórbida"
                color = 'darkred'

        
        except ValueError:
            return "Por favor valida los valores ingresados.", ""

        Age = rango_edad(edad)
        # Preparar los datos para la predicción
        input_data = np.array([[BMI, HighBP, HighChol, HeartDiseaseorAttack, GenHlth, Age]])

        # Usar el modelo para hacer la predicción
        try:
            prediccion_diabetes = modelo_diabetes.predict(input_data)  # Usar el modelo cargado
            if prediccion_diabetes == 1:
                prediccion_message = "Alto riesgo de diabetes."
                color1 = 'red'
            else:
                prediccion_message = "Bajo riesgo de diabetes."
                color1 = 'green'
        except Exception as e:
            prediccion_message = f"Error en la predicción: {str(e)}"
        
        return (html.Div(bmi_message, style={'color': color, 'fontSize': '24px'}),
                html.Div(prediccion_message, style={'color': color1, 'fontSize': '35px'}))

    return "Ingrese su estatura y peso para calcular el IMC.", ""


In [11]:
if __name__ == '__main__':
    app.run_server()